<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TestOUX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemple d'apprentissage du OUX avec TensorFlow


##Importer les librairires

In [0]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

---

***Définir les entrées et sorties attendues***


In [0]:
entrees = np.array([[0,0],[0,1],[1,0],[1,1]], float)
sorties = np.array([[0],[1],[1],[0]], float)

---

***Choisir le modèle de réseau*** (*ici les couches sont séquentielles*)

In [0]:
model = Sequential()

---

***Définir l'architecture du réseau*** 

In [0]:
avec_bias = True
#une premiere couche constituée de 2 neurones en sortie, de 3 neurones en entrée (2 pour les valeurs + 1 Bias), fonction d'activation = sigmoide
model.add(Dense(2, input_dim=2, use_bias=avec_bias, activation='sigmoid'))

#une seconde couche constituée de 1 neurone en sortie (et implicitement de 3 neurones en entrée), fonction d'activation = sigmoide
model.add(Dense(1, activation='sigmoid'))


---

***Compiler le  réseau*** * (ici, on précise que l'algo de correction d'erreur est 'adam', et que l'erreur calculée est la moyenne des valeurs absolues des erreurs commises)*

In [0]:
model.compile(optimizer='adam', loss='MSE')

---

***Entraîner le réseau*** (*ici on ne le fait pas parler, et on lance 10000 cycles d'apprentissage*)

In [6]:
model.fit(entrees, sorties, verbose=0, epochs=10000)

---

***Vérifier le réseau*** (*Etape facultative, en générale on le teste sur d'autres exemples. Ici, on lui demande de calculer la sortie pour chaque exemple de l'ensemble d'entraînement*)

In [0]:
predictions = model.predict(entrees)

---
**Affichage des résultats** (*on affiche les entrées, la sortie attendue, la sortie calculées ainsi que les poids appliquées aux entrées et au signal bias*) 

In [10]:
print("verification")
for i in range(0, len(entrees)):
    print(str(entrees[i][0]), " - ", entrees[i][1], " attendu ", sorties[i], " trouve ",  predictions[i])


verification
0.0  -  0.0  attendu  [0.]  trouve  [0.03521344]
0.0  -  1.0  attendu  [1.]  trouve  [0.9835434]
1.0  -  0.0  attendu  [1.]  trouve  [0.9576686]
1.0  -  1.0  attendu  [0.]  trouve  [0.0316713]
